In [235]:
import numpy as np
import gzip
import time
import glob
from sklearn.neighbors import NearestNeighbors
to=time.time()

fullcode={b"HIS":b"0",
        b"ASP":b"1",
        b"LYS":b"2",
        b"TYR":b"3",
        b"GLU":b"4",
        b"CYS":b"5"}
code={b"H":b"0",
        b"A":b"1",
        b"L":b"2",
        b"T":b"3",
        b"G":b"4",
        b"C":b"5"}
elements = {
    b"D":1, b"H": 1, b"LI":3, b"C": 6, b"N": 7, b"O": 8, 
    b"F":9,
    b"NA": 11, b"MG": 12,  b"P": 15, b"S": 16, b"CL": 17, b"K": 19,
    b"CA": 20, b"MN": 25, b"FE": 26, b"CO":27, b"NI":28,
    b"CU": 29, b"ZN": 30,
    b"SE":34,
    b"MO":42, b"SN":50, 
    b"I":53, b"CS":55,
    b"W":74, b"PT":78
    }
elements={k.decode() :np.int32(v) for k,v in elements.items()}

class pkparser():
    """#TODO save terminus files
    last id full path include dir. assumed XXXpdb.gz as is gotten from RCSB."""
    """it was necessary to use numpy, because I cant make jagged tensors."""
    def __init__(self,gzipped_pdb,cutoff):
        self.path = gzipped_pdb#"/Users/jessihoernschemeyer/pKaSchNet/pkas.csv" #('/content/drive/MyDrive/pkas.csv') #3directory
        self.pdb = gzipped_pdb[-11:-7]
        self.targets=np.load(f"/home/jrhoernschemeyer/Desktop/data_prep/targets/{self.pdb}.npz")
        self.cutoff=cutoff #Angstrom
        self.ligands=np.load("ligands.npz")["data"]

    def parse_titratable_lines(self,lines):
        """get info from asp,glu,his,cys,tyr. removes hydrogens, skips insertion codes, takes most probable config or "A" if 50-50."""
        pdbspecies, pdbcoors,terminus_lines=[],[],[]
        species,coors,line,lastresi,strii=None,None,None,None,None
        
        while lines:
            stri=lines[0]
            if not stri[0:1] == b"H":
                #alt conformations
                if not np.char.isspace(stri[3:4]):
                    strii=lines[1]
                    stri1,stri2=stri[43:47], strii[43:47]
                    if np.char.greater_equal(stri1,stri2):
                        resi=stri[4:14]
                        resinum=resi[5:].strip()

                        #insertion
                        if not resinum.isdigit(): 
                            self.others.append(lines[0])
                            del lines[0]
                            self.others.append(lines[0])
                            del lines[0]
                            strii=None
                            #TODO: put them in all_other_coors
                            continue 
                        
                        #continuing resi
                        if resinum==lastresi:
                            line, lastresi=stri,resinum
                            species.append(elements[list(filter(str.isalpha, line[-4:].decode()))[0]])
                            coors.append((np.float32(line[17:25]),np.float32(line[25:33]),np.float32(line[33:41])))
                            del lines[0]
                            del lines[0]
                        else: #newresi
                            pdbspecies.append(np.array(species))
                            pdbcoors.append(np.array(coors))
                            terminus_lines.append(line)
                            line,lastresi=stri,resi
                            species=[elements[list(filter(str.isalpha, line[-4:].decode()))[0]]]
                            coors=[(np.float32(line[17:25]),np.float32(line[25:33]),np.float32(line[33:41]))]
                            lastresi=resinum
                            del lines[0]
                            del lines[0]
                            strii=None     

                    else:
                            #B is greater
                        resi=strii[4:14]
                        resinum=resi[5:].strip()
                        if not resinum.isdigit(): #INSERT
                            self.others.append(lines[0])
                            del lines[0]
                            self.others.append(lines[0])
                            del lines[0]
                            continue

                        if resinum==lastresi:
                            line, lastresi=strii,resinum
                            species.append(elements[list(filter(str.isalpha, line[-4:].decode()))[0]])
                            coors.append((np.float32(line[17:25]),np.float32(line[25:33]),np.float32(line[33:41])))
                            del lines[0]
                        else:  #newresi
                            pdbspecies.append(np.array(species))
                            pdbcoors.append(np.array(coors))
                            terminus_lines.append(line)
                            line=strii
                            strii=None
                            species=[elements[list(filter(str.isalpha, line[-4:].decode()))[0]]]
                            coors=[(np.float32(line[17:25]),np.float32(line[25:33]),np.float32(line[33:41]))]
                            lastresi=resinum
                            del lines[0]

                #normal run
                else:
                    resi=stri[4:14]
                    resinum=resi[5:].strip()
                    if not resinum.isdigit(): #INSERTion
                        self.others.append(lines[0])
                        del lines[0]
                        self.others.append(lines[0])
                        del lines[0]

                    if resinum==lastresi:
                        line,lastresi=stri,resinum
                        species.append(elements[list(filter(str.isalpha, line[-4:].decode()))[0]])
                        coors.append((np.float32(line[17:25]),np.float32(line[25:33]),np.float32(line[33:41])))
                        del lines[0]
                    else: #newresi
                        pdbspecies.append(np.array(species))
                        pdbcoors.append(np.array(coors))
                        terminus_lines.append(line)
                        line,lastresi=stri,resinum
                        species=[elements[list(filter(str.isalpha, line[-4:].decode()))[0]]]
                        coors=[(np.float32(line[17:25]),np.float32(line[25:33]),np.float32(line[33:41]))]
                        lastresi=resinum
                        del lines[0]
            else: #hydrogens-skip
                del lines[0]
                continue

            
                     
        pdbspecies.append(np.array(species))
        pdbcoors.append(np.array(coors))
        terminus_lines.append(line)
        #TODO save terminus lines
        self.terminus_lines = terminus_lines[1:]
        self.ids=[np.char.add(np.char.add(np.char.strip(b[-5:]),b[-6:-5]),code[b[4:5]]) for b in [t[:14] for t in terminus_lines[1:]]]
        #return np.array(pdbspecies,dtype=object), np.array(pdbcoors,dtype=object)
        return pdbspecies, pdbcoors
    
    def aggregate_others(self):
        """#TODO"""
        #def stack_columns(data):
        """
        data: list of (int_arr, coord_arr) tuples, where
        - int_arr is either a 1D np.ndarray of ints or an object array of int sub-arrays
        - coord_arr is either a 2D np.ndarray of shape (M,3) or an object array of 2D sub-arrays
        Returns:
        - all_ints: 1D np.ndarray of all ints concatenated
        - all_coords: 2D np.ndarray of shape (total_rows, 3)
        """
        int_chunks = []
        coord_chunks = []

        for int_arr, coord_arr in self.others:
            # collect ints
            if int_arr.dtype == object:
                # flatten object-array of int sub-arrays
                for sub in int_arr:
                    int_chunks.append(np.ravel(sub))
            else:
                int_chunks.append(int_arr.ravel())

            # collect coords
            if coord_arr.dtype == object:
                # flatten object-array of 2D sub-arrays
                for sub in coord_arr:
                    coord_chunks.append(sub)
            else:
                coord_chunks.append(coord_arr)

        all_ints   = np.concatenate(int_chunks, axis=0)
        all_coords = np.concatenate(coord_chunks, axis=0)
        return all_ints, all_coords

    def parse_ligands(self,lines):
            """parse ligands #TODO"""
            species,coors=[],[]
            lines=[line[13:] for line in lines if line[16:20].strip() in self.ligands] 
            while lines:
                line=lines[0]
                if not line[0:1] == b"H":
                    #alt conformations
                    if np.char.isspace(line[3:4]):
                        strii=lines[1]
                        stri1,stri2=line[43:47], strii[43:47]
                        if np.char.greater_equal(stri1,stri2):
                            species.append(elements[list(filter(str.isalpha, line[-4:].decode()))[0]])
                            coors.append((np.float32(line[17:25]),np.float32(line[25:33]),np.float32(line[33:41])))
                            del lines[0]
                            del lines[0]
                              

                        else: 
                                #B is greater
                            species.append(elements[list(filter(str.isalpha, line[-4:].decode()))[0]])
                            coors.append((np.float32(line[17:25]),np.float32(line[25:33]),np.float32(line[33:41])))
                            del lines[0]


                    #normal run
                    else:
                        species.append(elements[list(filter(str.isalpha, line[-4:].decode()))[0]])
                        coors.append((np.float32(line[17:25]),np.float32(line[25:33]),np.float32(line[33:41])))
                        del lines[0]
                else: #hydrogens-skip
                    del lines[0]
                    continue

            self.others.append((np.array(species),np.array(coors)))

            #print(3,species,coors)
   
    def parse_others(self,lines):
        """"""
        species,coors=[],[]
        #ligands=np.load("ligands.npz")["data"]
        lines=[line[13:] for line in lines] 
        while lines:
            line=lines[0]
            if not line[0:1] == b"H":
                #alt conformations
                if np.char.isspace(line[3:4]):
                    strii=lines[1]
                    stri1,stri2=line[43:47], strii[43:47]
                    if np.char.greater_equal(stri1,stri2):
                        species.append(elements[list(filter(str.isalpha, line[-4:].decode()))[0]])
                        coors.append((np.float32(line[17:25]),np.float32(line[25:33]),np.float32(line[33:41])))
                        del lines[0]
                        del lines[0]
                            

                    else: 
                            #B is greater
                        species.append(elements[list(filter(str.isalpha, line[-4:].decode()))[0]])
                        coors.append((np.float32(line[17:25]),np.float32(line[25:33]),np.float32(line[33:41])))
                        del lines[0]


                #normal run
                else:
                    species.append(elements[list(filter(str.isalpha, line[-4:].decode()))[0]])
                    coors.append((np.float32(line[17:25]),np.float32(line[25:33]),np.float32(line[33:41])))
                    del lines[0]
            else: #hydrogens-skip
                del lines[0]
                continue

        self.others.append((np.array(species),np.array(coors)))

        #print(3,species,coors)
   
    def parse_pdb(self):
        """to do try except re: encoding/gzipped"""
        with gzip.open(self.path, "r") as f:
            lines=np.char.array(f.readlines())
            f.close()
        #encode everything if user didnt gzip their filess

        #run
       
        lines=np.char.array([line for line in lines if np.char.startswith(line,[b"ATOM",b"HETATM"]).any()])
        print(len(lines))
        hets=[(i,line) for i,line in enumerate(lines) if np.char.startswith(line,b"HETATM")]
        lines=np.delete(lines,[idx[0] for idx in hets])
        self.parse_ligands([a[1] for a in hets])
        #lines=np.char.array([line for line in lines if np.char.startswith(line,b"ATOM")])#for line in lines if np.char.startswith(line,"ATOM")])
        tlines=[(i,line[13:]) for i,line in enumerate(lines) if line[16:20].strip() in fullcode.keys()]
        self.parse_others(np.delete(lines,[idx[0] for idx in tlines]))
        #tlines=[line[1] for line in tlines]
        pdbspecies,pdbcoors=self.parse_titratable_lines([line[1] for line in tlines])
        
        #self.parse_regular_lines()


        return np.array(pdbspecies[1:],dtype=object), np.array(pdbcoors[1:],dtype=object)
    

    def hoods(self,coors,species):
        others=self.aggregate_others()
        all_coors=np.vstack([*coors,others[1]])
        neigh = NearestNeighbors(algorithm="brute")
        neigh.fit(all_coors)
        idxs,last=[],0
        for z in species:
            idxs.append(len(z) + last - 1)
            last=idxs[-1]
        nbrs = neigh.radius_neighbors(all_coors[idxs],radius=self.cutoff, return_distance=False)
        
        species=np.concatenate([*[*species, others[0]]]) #allspecoes
        return [species[n_ixs] for n_ixs in nbrs], [all_coors[n_ixs].astype(np.float32) for n_ixs in nbrs]
    
    
    def get_disulfides(self,terminus_lines):
        """#TODO check for sulfur?"""

        cys_lines = [(i,line) for i,line in enumerate(terminus_lines) if line[4:5] == b"C"]
        cys_coors = [(np.float32(line[1][17:25]),np.float32(line[1][25:33]),np.float32(line[1][33:41])) for line in cys_lines] #TODO confirm its S?

        neigh = NearestNeighbors(algorithm="brute")
        neigh.fit(cys_coors)
        nbrs = neigh.radius_neighbors(cys_coors,radius=2.05, return_distance=False)
        bridges = (np.array([cys[0] for cys in cys_lines]),np.array([len(a) == 2 for a in nbrs]))
        self.disulfides=bridges[0][bridges[1]]
        

    def run(self):
        """TODO save terminus lines"""
        self.others=[]

        #get structures of titratable residues
        species,coors=self.parse_pdb()
        #parseligands

        #get targets
        pkpdb=self.targets
        ids, pidx, midx = np.intersect1d(pkpdb["ids"], self.ids,return_indices=True)

        #get unmatched structures
        mask = np.ones(len(self.ids), dtype=bool)
        mask[midx] = False
        if mask.any():
            self.others.append((species[mask], coors[mask]))
        
        #disulfide mask
        self.get_disulfides(self.terminus_lines) #TODO Integrate with all_non_tit_coors matrix machen

        if self.disulfides.any(): 
            sulf=self.disulfides
            #delete from ids
            self.ids=np.delete(self.ids,self.disulfides)
            #save other lines
            self.others.append((species[sulf], coors[sulf]))
      
        
        #get titratable cuts with labeled structures
        #TODO put other atoms and ligands in
        species,coors=self.hoods(coors[midx],species[midx])


        #save input
        np.savez_compressed(f"/home/jrhoernschemeyer/Desktop/data_prep/{self.pdb}.npz",z=species, pos=coors, pks=pkpdb["pks"][pidx],ids=ids)

        return self
        #except:
        #"zip the pdb or find pdb code somehow else lol or seperate home dir from after"
        
me=pkparser(gzipped_pdb=f"/home/jrhoernschemeyer/Desktop/data_prep/structures/pdbs/4b00.pdb.gz",cutoff=5).run()
[len(x) for x in np.load("/home/jrhoernschemeyer/Desktop/data_prep/4b00.npz",allow_pickle=True)["z"]]

3390


/home/jrhoernschemeyer/.local/lib/python3.6/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


[8,
 15,
 12,
 21,
 27,
 15,
 18,
 22,
 15,
 12,
 12,
 9,
 8,
 28,
 21,
 18,
 22,
 18,
 24,
 20,
 18,
 25,
 15,
 22,
 18,
 21,
 15,
 21,
 16,
 18,
 12,
 27,
 10,
 31,
 26,
 13,
 18,
 17,
 20,
 16,
 16,
 19,
 18,
 23,
 23,
 17,
 19,
 12,
 21,
 24,
 10,
 19,
 16,
 12,
 12,
 39,
 32,
 21,
 16,
 22,
 26,
 26,
 19,
 18,
 12,
 19,
 19,
 23,
 13,
 16,
 12,
 22,
 10,
 14,
 19,
 10,
 18,
 21,
 20,
 19,
 24,
 22,
 22]

In [12]:
with open("/home/jrhoernschemeyer/Documents/lig2pdb.lst", "r") as f:
    lines=np.char.encode(np.char.array(f.readlines()))
    
c=[b.strip() for b in lines if not np.char.startswith(b,b" ")]
ligands = set().union(*[*[set(a.split(b"-")) for a in c]])

In [73]:
np.char.array(list(ligands))

chararray([b'56X', b'6B9', b'8O4', ..., b'B19', b'42G', b'XZY'],
          dtype='|S9')

In [74]:
np.savez_compressed("ligands.npz",data=np.char.array(list(ligands)))